<b><h1><font color = "red">Bartys Test NoteBook.</font> </h1></b> </br> Contains tests. Not to be run within ROS.

In [1]:
import numpy as np
import cv2

In [2]:
img = cv2.imread('Refference Images/WithRedDot/Grid1.jpg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [3]:


#The Blue mask
lower_blue = np.array([90,130,80])
upper_blue = np.array([115,255,255])

#The Yellow Mask
lower_yellow = np.array([20,204,150])
upper_yellow = np.array([54,255,255])

#The red mask
lower_red = np.array([0,110,139])
upper_red = np.array([10,255,255])
maskr = cv2.inRange(hsv, lower_red, upper_red)

'''
# upper mask (170-180)
lower_red = np.array([170,10,10])
upper_red = np.array([180,255,255])
maskr2 = cv2.inRange(hsv, lower_red, upper_red)

'''


# Threshold the HSV image to get only blue colors
maskb = cv2.inRange(hsv, lower_blue, upper_blue)
masky = cv2.inRange(hsv, lower_yellow, upper_yellow) #Needs tweeaking
maskr = cv2.inRange(hsv, lower_red, upper_red)


In [ ]:
#thresh= cv2.adaptiveThreshold(masky , 255 ,cv2.ADAPTIVE_THRESH_GAUSSIAN_C , cv2.THRESH_BINARY , 115,1)
contours , hierachy = cv2.findContours(maskr , cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE)

img2 = img.copy()
index = -1
thickness = 10
colour = (255 , 0 , 255)
cv2.drawContours(img2,contours , index , colour , thickness)


cv2.namedWindow("image3", cv2.WINDOW_NORMAL)
cv2.imshow('image3',img2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
#for finding the 4 Red Centres.

Red_cordinates = []

for c in contours:
    area = cv2.contourArea(c)
    if area > 500:
        M = cv2.moments(c)
        try:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            print(area)
            Red_cordinates.append([cX ,cY])
        except ZeroDivisionError:
            pass


reorder = [3,2,1,0]
Red_cordinates= [Red_cordinates[i] for i in reorder]
print(Red_cordinates)

1702.5
1701.0
1624.5
1610.5
[[648, 456], [2971, 635], [928, 1715], [2764, 2091]]


In [ ]:
pts_dst = np.array([[0,0] , [700,0] , [700,600] ,[0 , 600]],np.float32)
Red_cordinates = np.array(Red_cordinates,np.float32)

#h, status = cv2.findHomography(pts_src, Red_cordinates)
h = cv2.getPerspectiveTransform(Red_cordinates,pts_dst)
im_out = cv2.warpPerspective(img, h,(700,600))

cv2.namedWindow("image12", cv2.WINDOW_NORMAL)
cv2.imshow('image12',im_out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
stack = np.hstack((maskb, masky,maskr))
cv2.namedWindow("image", cv2.WINDOW_NORMAL)
cv2.imshow('image',stack)
cv2.namedWindow("image2", cv2.WINDOW_NORMAL)
cv2.imshow('image2',img)

cv2.namedWindow("image3", cv2.WINDOW_NORMAL)
cv2.imshow('image2',thresh)

cv2.waitKey(0)
cv2.destroyAllWindows()


<h1> Not my code but its very good for calibarition </h1>

In [12]:
import cv2
import sys
import numpy as np

def nothing(x):
    pass

# Load in image
image = cv2.imread('Refference Images/WithRedDot/Grid1.jpg')

# Create a window
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

# create trackbars for color change
cv2.createTrackbar('HMin','image',0,179,nothing) # Hue is from 0-179 for Opencv
cv2.createTrackbar('SMin','image',0,255,nothing)
cv2.createTrackbar('VMin','image',0,255,nothing)
cv2.createTrackbar('HMax','image',0,179,nothing)
cv2.createTrackbar('SMax','image',0,255,nothing)
cv2.createTrackbar('VMax','image',0,255,nothing)

# Set default value for MAX HSV trackbars.
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize to check if HSV min/max value changes
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

output = image
wait_time = 33

while(1):

    # get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin','image')
    sMin = cv2.getTrackbarPos('SMin','image')
    vMin = cv2.getTrackbarPos('VMin','image')

    hMax = cv2.getTrackbarPos('HMax','image')
    sMax = cv2.getTrackbarPos('SMax','image')
    vMax = cv2.getTrackbarPos('VMax','image')

    # Set minimum and max HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Create HSV Image and threshold into a range.
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    output = cv2.bitwise_and(image,image, mask= mask)

    # Print if there is a change in HSV value
    if( (phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display output image
    cv2.imshow('image',output)

    # Wait longer to prevent freeze for videos.
    if cv2.waitKey(wait_time) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

(hMin = 0 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 15 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 45 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 64 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 71 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 83 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 86 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 87 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 91 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 97 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 103 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 111 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 117 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 124 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, v